In [39]:
# !pip install -U FlagEmbedding[finetune]

In [40]:
import os
os.environ['WANDB_DISABLED'] = 'true'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [41]:
!pip install gdown

In [42]:
!gdown 1anFFKuEoL97bggwrYj8On-QciMIU_2l1

Downloading...
From: https://drive.google.com/uc?id=1anFFKuEoL97bggwrYj8On-QciMIU_2l1
To: /kaggle/working/data_full_spoiled.json
100%|███████████████████████████████████████| 1.52M/1.52M [00:00<00:00, 152MB/s]


In [43]:
!gdown 1yWBShb9wV6kBVVCzNSaO5LFu5_faToQg

Downloading...
From: https://drive.google.com/uc?id=1yWBShb9wV6kBVVCzNSaO5LFu5_faToQg
To: /kaggle/working/label_desc_context_new.csv
100%|██████████████████████████████████████| 33.8k/33.8k [00:00<00:00, 57.6MB/s]


In [44]:
!gdown 1UlYk6XdzwK4U1AuPvmBv5i1Odx7K9puZ

Downloading...
From: https://drive.google.com/uc?id=1UlYk6XdzwK4U1AuPvmBv5i1Odx7K9puZ
To: /kaggle/working/label_desc_context_nearest_many.csv
100%|██████████████████████████████████████| 41.5k/41.5k [00:00<00:00, 59.4MB/s]


In [45]:
import json

with open("data_full_spoiled.json", "r") as f:
    data = json.load(f)

In [46]:
import pandas as pd

desc = pd.read_csv("label_desc_context_nearest_many.csv")
desc

,intent,text,description,nearest_labels
0,accept_reservations,does spago in beverly hills allow customers to...,Requests to determine if a specific restaurant...,"['definition', 'travel_alert', 'restaurant_res..."
1,account_blocked,my account looks to be blocked and i have no i...,Requests for information about why a bank acco...,"['balance', 'freeze_account', 'card_declined']"
2,alarm,i'd like alarms set for 11am and 132pm,"Requests to set, update, or review alarms for ...","['translate', 'timer']"
3,application_status,what is the present status of my credit card a...,Requests for updates on the status of an appli...,"['credit_limit', 'order_status', 'credit_limit..."
4,apr,is the apr on my card that ends 1234 what it s...,Requests for information about the Annual Perc...,"['new_card', 'credit_limit', 'interest_rate']"
...,...,...,...,...
146,where_are_you_from,what place were you born at,Requests about the assistant's origin or creat...,"['change_user_name', 'what_can_i_ask_you', 'wh..."
147,whisper_mode,set up whisper mode,"Requests to activate a whisper mode, where the...","['change_user_name', 'change_speed', 'change_v..."
148,who_do_you_work_for,do you work for another person,"Requests to know the assistant's affiliation, ...","['application_status', 'change_user_name', 'wh..."
149,who_made_you,who is your designer,"Requests about the creator, designer, or under...","['fun_fact', 'change_user_name', 'where_are_yo..."


In [47]:
label2desc = {row.intent: row.description for i, row in desc.iterrows() if row.intent != "oos"}
list(label2desc.items())[:5]

[('accept_reservations',
  'Requests to determine if a specific restaurant or establishment allows reservations. Users may seek this information before attempting to book a table.'),
 ('account_blocked',
  'Requests for information about why a bank account or service has been blocked. Users may seek assistance in understanding the reason for the hold and how to resolve it.'),
 ('alarm',
  'Requests to set, update, or review alarms for specific times. Users may provide detailed instructions for single or multiple alarms across different time periods.'),
 ('application_status',
  'Requests for updates on the status of an application, such as a credit card or other formal process. Users may ask if their application has been approved, is under review, or requires additional action.'),
 ('apr',
  'Requests for information about the Annual Percentage Rate (APR) on a credit card. Users may ask if the rate is appropriate based on their credit score or inquire about the APR for a specific card.

In [48]:
label2nearest = {row.intent: row.nearest_labels[1:-1].split(",")[0][1:-1] for i, row in desc.iterrows() if row.intent != "oos"}
list(label2nearest.items())[:5]

[('accept_reservations', 'definition'),
 ('account_blocked', 'balance'),
 ('alarm', 'translate'),
 ('application_status', 'credit_limit'),
 ('apr', 'new_card')]

# Data

In [49]:
import random

train_data = {"anchor": [], "positive": [], "negative": []}
for text, label in data["train"]:
    train_data["anchor"].append(text)
    train_data["positive"].append(label2desc[label])
    train_data["negative"].append(label2desc[label2nearest[label]])
    # neg_label = random.randint(0, len(label2desc) - 1)
    # while neg_label == label:
    #     neg_label = random.randint(0, len(label2desc) - 1)        
    # train_data["negative"].append(label2desc[neg_label])

In [50]:
with open("train_data.json", "w") as f:
    json.dump(train_data, f, indent=2)

In [51]:
from IPython.display import FileLink 

FileLink(r'train_data.json')

/kaggle/working/train_data.json

In [52]:
from datasets import Dataset
train_dataset = Dataset.from_dict(
    train_data
)

In [53]:
val_data = {"anchor": [], "positive": []}
for text, label in data["val"]:
    val_data["anchor"].append(text)
    val_data["positive"].append(label2desc[label])
    # val_data["negative"].append(label2desc[label2nearest[label]])    
    # while neg_label == label:
    #     neg_label = random.randint(0, len(label2desc) - 1)        
    # val_data["negative"].append(label2desc[neg_label])

In [54]:
with open("val_data.json", "w") as f:
    json.dump(val_data, f, indent=2)

In [55]:
with open("label2desc.json", "w") as f:
    json.dump(label2desc, f, indent=2)

In [56]:
from IPython.display import FileLink 

FileLink(r'val_data.json')

/kaggle/working/val_data.json

In [57]:
from IPython.display import FileLink 

FileLink(r'label2desc.json')

/kaggle/working/label2desc.json

In [58]:
from datasets import Dataset
val_dataset = Dataset.from_dict(
    val_data
)

In [59]:
len(train_data["anchor"]), len(val_data["anchor"])

(15002, 3000)

In [60]:
# train_dataset = val_dataset

# Model

In [61]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.3 MB/s eta 0:00:00:00:01


In [62]:
from sentence_transformers import SentenceTransformer

model_id = "BAAI/bge-large-en-v1.5"

model = SentenceTransformer(model_id)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [63]:
from sentence_transformers.losses import MultipleNegativesRankingLoss
# from sentence_transformers.losses import TripletLoss

train_loss = MultipleNegativesRankingLoss(model)
# train_loss = TripletLoss(model)

In [64]:
from sentence_transformers import SentenceTransformerTrainingArguments

args = SentenceTransformerTrainingArguments(
    output_dir="bge",
    num_train_epochs=4,
    per_device_train_batch_size=32, 
    warmup_ratio=0.2,     
    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    optim="adamw_torch_fused",
    # fp16=True, 
    # eval_strategy="epoch",
    # save_strategy="epoch",
    eval_strategy="steps",
    eval_steps=150,
    save_strategy="steps",
    save_steps=150,
    save_total_limit=3,
    load_best_model_at_end=True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [65]:
from sentence_transformers.evaluation import TripletEvaluator, InformationRetrievalEvaluator, SequentialEvaluator
from sentence_transformers.util import cos_sim

# triplet_evaluator = TripletEvaluator(
#     anchors=val_dataset["anchor"],
#     positives=val_dataset["positive"],
#     negatives=val_dataset["negative"],
# )

desc2id = {v: i for i, (k, v) in enumerate(label2desc.items())}
id2desc = {i: v for i, (k, v) in enumerate(label2desc.items())}

corpus = dict(
    [(desc2id[v], v) for i, (k, v) in enumerate(label2desc.items())]
)

queries = dict(
    zip(list(range(len(val_dataset["anchor"]))), val_dataset["anchor"])
)
 
relevant_docs = {}
for q_id in range(len(val_dataset["anchor"])):
    relevant_docs[q_id] = [desc2id[val_dataset["positive"][q_id]]]

information_retrieval_evaluator = InformationRetrievalEvaluator(
    queries=queries,
    corpus=corpus,
    relevant_docs=relevant_docs,
    score_functions={"cosine": cos_sim}
)

evaluator = SequentialEvaluator([information_retrieval_evaluator])

In [66]:
len(queries), len(corpus), len(relevant_docs)

(3000, 150, 3000)

In [67]:
from sentence_transformers import SentenceTransformerTrainer

trainer = SentenceTransformerTrainer(
    model=model,
    args=args, 
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    loss=train_loss,
    evaluator=evaluator
    # compute_metrics=compute_metrics
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [68]:
trainer.evaluate()

{'eval_loss': 0.27297139167785645,
 'eval_model_preparation_time': 0.0083,
 'eval_cosine_accuracy@1': 0.8016666666666666,
 'eval_cosine_accuracy@3': 0.952,
 'eval_cosine_accuracy@5': 0.9733333333333334,
 'eval_cosine_accuracy@10': 0.9876666666666667,
 'eval_cosine_precision@1': 0.8016666666666666,
 'eval_cosine_precision@3': 0.31733333333333336,
 'eval_cosine_precision@5': 0.19466666666666663,
 'eval_cosine_precision@10': 0.09876666666666666,
 'eval_cosine_recall@1': 0.8016666666666666,
 'eval_cosine_recall@3': 0.952,
 'eval_cosine_recall@5': 0.9733333333333334,
 'eval_cosine_recall@10': 0.9876666666666667,
 'eval_cosine_ndcg@10': 0.9054911301597135,
 'eval_cosine_mrr@10': 0.8779052910052914,
 'eval_cosine_map@100': 0.8785822680177376,
 'eval_sequential_score': 0.9054911301597135,
 'eval_runtime': 31.8273,
 'eval_samples_per_second': 94.259,
 'eval_steps_per_second': 11.782}

In [69]:
trainer.train()

Step,Training Loss,Validation Loss,Model Preparation Time,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100,Sequential Score
150,No log,0.069754,0.008300,0.920333,0.981333,0.992000,0.996667,0.920333,0.327111,0.198400,0.099667,0.920333,0.981333,0.992000,0.996667,0.963284,0.952042,0.952197,0.963284
300,No log,0.064207,0.008300,0.933333,0.982333,0.989000,0.996667,0.933333,0.327444,0.197800,0.099667,0.933333,0.982333,0.989000,0.996667,0.968323,0.958893,0.959057,0.968323
450,No log,0.060303,0.008300,0.951000,0.987667,0.991667,0.995667,0.951000,0.329222,0.198333,0.099567,0.951000,0.987667,0.991667,0.995667,0.976324,0.969807,0.970059,0.976324
600,1.033800,0.061176,0.008300,0.951667,0.986333,0.991000,0.995333,0.951667,0.328778,0.198200,0.099533,0.951667,0.986333,0.991000,0.995333,0.976205,0.969799,0.970011,0.976205
750,1.033800,0.060166,0.008300,0.962333,0.987667,0.991333,0.994333,0.962333,0.329222,0.198267,0.099433,0.962333,0.987667,0.991333,0.994333,0.980161,0.975415,0.975682,0.980161
900,1.033800,0.057140,0.008300,0.965667,0.988333,0.991667,0.995333,0.965667,0.329444,0.198333,0.099533,0.965667,0.988333,0.991667,0.995333,0.982048,0.977638,0.977862,0.982048
1050,0.787000,0.058461,0.008300,0.967667,0.986333,0.990667,0.994333,0.967667,0.328778,0.198133,0.099433,0.967667,0.986333,0.990667,0.994333,0.981870,0.977785,0.978075,0.981870
1200,0.787000,0.056539,0.008300,0.970667,0.988667,0.992000,0.995667,0.970667,0.329556,0.198400,0.099567,0.970667,0.988667,0.992000,0.995667,0.984196,0.980417,0.980605,0.984196
1350,0.787000,0.057824,0.008300,0.971667,0.988667,0.991000,0.994000,0.971667,0.329556,0.198200,0.099400,0.971667,0.988667,0.991000,0.994000,0.983718,0.980329,0.980637,0.983718
1500,0.676800,0.056994,0.008300,0.972333,0.988667,0.991333,0.995000,0.972333,0.329556,0.198267,0.099500,0.972333,0.988667,0.991333,0.995000,0.984435,0.980976,0.981202,0.984435


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=1876, training_loss=0.7923307683167935, metrics={'train_runtime': 2704.9295, 'train_samples_per_second': 22.185, 'train_steps_per_second': 0.694, 'total_flos': 0.0, 'train_loss': 0.7923307683167935, 'epoch': 4.0})

In [70]:
model.eval()
model.save_pretrained("best")

In [78]:
from huggingface_hub import notebook_login

notebook_login()

In [79]:
model.push_to_hub("bge-finetuned-train")

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

'https://huggingface.co/chinchilla04/bge-finetuned-train/commit/f36f8484dc176e47a246cd2f7ed7ca700dff1d94'

In [71]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args, 
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    loss=train_loss,
    evaluator=evaluator
    # compute_metrics=compute_metrics
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [72]:
trainer.evaluate()

{'eval_loss': 0.05653906241059303,
 'eval_model_preparation_time': 0.0065,
 'eval_cosine_accuracy@1': 0.9706666666666667,
 'eval_cosine_accuracy@3': 0.9886666666666667,
 'eval_cosine_accuracy@5': 0.992,
 'eval_cosine_accuracy@10': 0.9956666666666667,
 'eval_cosine_precision@1': 0.9706666666666667,
 'eval_cosine_precision@3': 0.3295555555555556,
 'eval_cosine_precision@5': 0.19840000000000002,
 'eval_cosine_precision@10': 0.09956666666666668,
 'eval_cosine_recall@1': 0.9706666666666667,
 'eval_cosine_recall@3': 0.9886666666666667,
 'eval_cosine_recall@5': 0.992,
 'eval_cosine_recall@10': 0.9956666666666667,
 'eval_cosine_ndcg@10': 0.9841961906084298,
 'eval_cosine_mrr@10': 0.9804173280423282,
 'eval_cosine_map@100': 0.9806052445247627,
 'eval_sequential_score': 0.9841961906084298,
 'eval_runtime': 30.9144,
 'eval_samples_per_second': 97.042,
 'eval_steps_per_second': 12.13}

In [73]:
!zip -r best.zip best

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  adding: best/ (stored 0%)
  adding: best/special_tokens_map.json (deflated 80%)
  adding: best/tokenizer.json (deflated 71%)
  adding: best/1_Pooling/ (stored 0%)
  adding: best/1_Pooling/config.json (deflated 57%)
  adding: best/README.md (deflated 73%)
  adding: best/tokenizer_config.json (deflated 75%)
  adding: best/config_sentence_transformers.json (deflated 35%)
  adding: best/vocab.txt (deflated 53%)
  adding: best/sentence_bert_config.json (deflated 4%)
  adding: best/2_Normalize/ (stored 0%)
  adding: best/model.safetensors (deflated 10%)
  adding: best/modules.json (deflated 62%)
  adding: best/config.json (deflated 47%)


In [74]:
from IPython.display import FileLink 

FileLink(r'best.zip')

/kaggle/working/best.zip

In [75]:
trainer.evaluate()

{'eval_loss': 0.05653906241059303,
 'eval_model_preparation_time': 0.0065,
 'eval_cosine_accuracy@1': 0.9706666666666667,
 'eval_cosine_accuracy@3': 0.9886666666666667,
 'eval_cosine_accuracy@5': 0.992,
 'eval_cosine_accuracy@10': 0.9956666666666667,
 'eval_cosine_precision@1': 0.9706666666666667,
 'eval_cosine_precision@3': 0.3295555555555556,
 'eval_cosine_precision@5': 0.19840000000000002,
 'eval_cosine_precision@10': 0.09956666666666668,
 'eval_cosine_recall@1': 0.9706666666666667,
 'eval_cosine_recall@3': 0.9886666666666667,
 'eval_cosine_recall@5': 0.992,
 'eval_cosine_recall@10': 0.9956666666666667,
 'eval_cosine_ndcg@10': 0.9841961906084298,
 'eval_cosine_mrr@10': 0.9804173280423282,
 'eval_cosine_map@100': 0.9806052445247627,
 'eval_sequential_score': 0.9841961906084298,
 'eval_runtime': 31.2335,
 'eval_samples_per_second': 96.051,
 'eval_steps_per_second': 12.006}